# dnd experiment to extract spells from a webpage

In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# Fancy url getter 

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [ ]:
# Request all pages

raw_html = simple_get('http://engl393-dnd5th.wikia.com/wiki/Wizard_Spells')

soup = BeautifulSoup(raw_html, 'html.parser');

df = pd.DataFrame()

spell_section = soup.find("div", {"id": "mw-content-text"})

for spells_by_level in spell_section.find_all("ul",recursive=False):
    spell_level = spells_by_level.find_previous_sibling('h2').text
    for spell in spells_by_level.find_all("li",recursive=False):
        df_temp = pd.DataFrame([[spell.a.text, spell_level, spell.a.attrs['href']]], columns=['name', 'level-name', 'url'])
        df = df.append(df_temp, ignore_index=True)

def get_detail_page(url):
    
    full_url = 'http://engl393-dnd5th.wikia.com' + url
    
    soup = BeautifulSoup(simple_get(full_url), 'html.parser')
    return soup.find("div", {"id": "mw-content-text"})

df['details'] = df['url'].map(get_detail_page)

In [ ]:
def level(level_name):
    return re.findall('\d+', level_name )[0]

df['level'] = df['level-name'].map(level)

def is_table_description(details):
    if(details.table == None):
        return False
    return details.table.find('th', text=re.compile('Casting.*')) != None

df['isTableDescription'] = df['details'].map(is_table_description)

def school(details):
    return details.find("i").text \
        .split(' cantrip')[0] \
        .split(' (ritual)')[0] \
        .split()[-1] \
        .lower()
    
df['school'] = df['details'].map(school)

def casting_time(details):
    try:
        if(is_table_description(details)):
            full_text = details.find('th', text=re.compile('Casting.*')).findNext('td').text
        else:
            full_text = details.find('b', text=re.compile('Casting.*')).parent.find(text=True, recursive=False)
        return full_text.strip(':').strip('\n').strip('\xa0').strip()
    except AttributeError as err:
        return err

df['castingTime'] = df['details'].map(casting_time)

In [ ]:
df.school.unique()

In [ ]:
df.castingTime.unique()

In [ ]:
df.loc[df['school'] == 'enchantment']